In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!ls ./data

Contec_device.ods  contec_eye.ods  Measurments_22_09_2017.ods  RealValues.ods


In [197]:
data = pd.read_csv('data/full_data.csv')

In [200]:
indicators = data.columns[1:11]

In [201]:
real = data[data.Device == 'Sysmex'][indicators].values

In [203]:
chinese = data[data.Device == 'chinese_device'][indicators].values

Будем тестировать точные выборки с помощью биномиального теста. Т.е. будем использовать тест для доли и сравнивать гипотезы о:

1) Отличается ли предсказание точного класса от случайного

2) Отличается ли предсказание с различием в один класс от случайного

3) Потом можно сравнить точности нескольких алгоритмов

! Везде нужно делать поправку на множественные сравнения (будем делать методом Бенджамини - Хохберга)

# Сравниваем точные показания Sysmex и Китайского прибора

## Hard accuracy с помощью биномиального теста

In [204]:
import scipy.stats as stats

In [205]:
p_vals = []
for i,ind in enumerate(indicators):
    sample = 1.*(real[:,i] == chinese[:,i])

    n = len(sample)
    x = np.sum(sample)
    p_vals.append(stats.binom_test(x,n=n,alternative='greater'))
p_vals = np.array(p_vals)

In [206]:
indicators[p_vals<0.05]

Index([u'GLU', u'NIT'], dtype='object')

In [207]:
import statsmodels.stats.multitest as ssm

In [208]:
print p_vals

[ 0.36328125  0.14453125  0.14453125  0.63671875  0.14453125  0.03515625
  0.85546875  0.96484375  0.03515625  0.99609375]


In [209]:
ssm.multipletests(p_vals,method='fdr_bh')

(array([False, False, False, False, False, False, False, False, False, False], dtype=bool),
 array([ 0.60546875,  0.2890625 ,  0.2890625 ,  0.90959821,  0.2890625 ,
         0.17578125,  0.99609375,  0.99609375,  0.17578125,  0.99609375]),
 0.0051161968918237433,
 0.005)

**Промежуточные выводы**: 
- Статистически значимые отличия биномиальным тестом на таком размере выборки c учётом множественной проверки гипотез подтвердить не удаётся. 
- Hard_accuracy китайского прибора в сравнении с sysmex мала

In [210]:
np.sum(real == chinese,axis=0)/float(real.shape[0])

array([ 0.625,  0.75 ,  0.75 ,  0.5  ,  0.75 ,  0.875,  0.375,  0.25 ,
        0.875,  0.125])

In [211]:
print indicators

Index([u'URO', u'BLO', u'BIL', u'KET', u'LEU', u'GLU', u'PRO', u'PH', u'NIT',
       u'SG'],
      dtype='object')


## Soft accuracy с помощью биномиального теста

In [212]:
p_vals = []
for i,ind in enumerate(indicators):
    sample = np.abs(real[:,i] - chinese[:,i]) <= 1

    n = len(sample)
    x = np.sum(sample)
    p_vals.append(stats.binom_test(x,n=n,alternative='greater'))
p_vals = np.array(p_vals)

### Индикаторы, для которых работает "отличие в один класс" 

In [213]:
indicators[ssm.multipletests(p_vals,method='fdr_bh')[0]]

Index([u'URO', u'BIL', u'KET', u'LEU', u'GLU', u'PRO', u'PH', u'NIT'], dtype='object')

In [214]:
print "Soft точности"
soft_accs = np.sum(np.abs(real - chinese)<=1,axis=0)/float(real.shape[0])
for ind,acc in zip(indicators,soft_accs):
    print ind,acc,' ', 

Soft точности
URO 0.875   BLO 0.75   BIL 1.0   KET 0.875   LEU 0.875   GLU 0.875   PRO 0.875   PH 0.875   NIT 0.875   SG 0.5  


# Cравниваем показания телефонов между собой

In [223]:
phones = ['Samsung','XiaomiB','XiaomiD']
template_types = ['inkjet','laser']

In [224]:
from itertools import combinations

In [249]:
data = data.sort_values(by='ID')

In [269]:
stripe_type = 'contec'

print "Тип полоски: ",stripe_type

for templ in template_types:
    for phone_i, phone_j in combinations(phones,2):
        x = data[(data.Device==phone_i)& (data.Template == templ)&(data.Strip==stripe_type)].iloc[:,:-2]
        y = data[(data.Device==phone_j)&(data.Template == templ)&(data.Strip==stripe_type)].iloc[:,:-2]

#         print phone_i
#         print x 
#         print phone_j
#         print y
#         print indicators.values
        print "Тип шаблона: ",templ, "Сравниваемые телефоны: ", phone_i, phone_j
        print "Средняя hard-точность ",
        print np.mean(np.sum(x[indicators].values == y[indicators].values,axis=0)/float(x.shape[0]))
        
        print '++++++++++ \n'

Тип полоски:  contec
Тип шаблона:  inkjet Сравниваемые телефоны:  Samsung XiaomiB
Средняя hard-точность  0.6125
++++++++++ 

Тип шаблона:  inkjet Сравниваемые телефоны:  Samsung XiaomiD
Средняя hard-точность  0.6625
++++++++++ 

Тип шаблона:  inkjet Сравниваемые телефоны:  XiaomiB XiaomiD
Средняя hard-точность  0.7
++++++++++ 

Тип шаблона:  laser Сравниваемые телефоны:  Samsung XiaomiB
Средняя hard-точность  0.675
++++++++++ 

Тип шаблона:  laser Сравниваемые телефоны:  Samsung XiaomiD
Средняя hard-точность  0.7125
++++++++++ 

Тип шаблона:  laser Сравниваемые телефоны:  XiaomiB XiaomiD
Средняя hard-точность  0.825
++++++++++ 



In [275]:
print "Тип полоски: ",stripe_type

for templ in template_types:
    for phone_i, phone_j in combinations(phones,2):
        x = data[(data.Device==phone_i)& (data.Template == templ)&(data.Strip==stripe_type)].iloc[:,:-2]
        y = data[(data.Device==phone_j)&(data.Template == templ)&(data.Strip==stripe_type)].iloc[:,:-2]

#         print phone_i
#         print x 
#         print phone_j
#         print y
#         print indicators.values
        print templ, phone_i, phone_j
        print "Средняя soft-точность ",
        print np.mean(np.sum(np.abs(x[indicators].values - y[indicators].values)<=1,axis=0)/float(x.shape[0]))
        
        print '++++++++++ \n'

Тип полоски:  contec
inkjet Samsung XiaomiB
Средняя soft-точность  0.825
++++++++++ 

inkjet Samsung XiaomiD
Средняя soft-точность  0.9375
++++++++++ 

inkjet XiaomiB XiaomiD
Средняя soft-точность  0.9
++++++++++ 

laser Samsung XiaomiB
Средняя soft-точность  0.9
++++++++++ 

laser Samsung XiaomiD
Средняя soft-точность  0.95
++++++++++ 

laser XiaomiB XiaomiD
Средняя soft-точность  0.9375
++++++++++ 



#### Выводы: даже показания с различных телефонов отличаются между собой.

Возможные причины: 
- изменение цвета индикатора на полоске за время между фотографиями (?)
- деградация шаблона
- реально различная цветопередача телефонов (как же так)
- неодинаковая неоднородная затенненность


***НУЖНО ПОСМОТРЕТЬ НА ФОТОГРАФИИ И ПРЕДСКАЗАНИЯ!***

# Сравниваем показания глаза и телефона

In [279]:
eye_data = data[data.Device == 'eye'][indicators].values

In [285]:
print eye_data

[[ 1.  0.  0.  3.  0.  0.  1.  5.  0.  3.]
 [ 0.  0.  0.  2.  0.  0.  0.  1.  0.  6.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  0.  6.]
 [ 0.  0.  2.  0.  0.  0.  1.  1.  0.  6.]
 [ 1.  0.  2.  0.  0.  0.  1.  1.  0.  6.]
 [ 0.  4.  2.  0.  0.  0.  0.  1.  0.  3.]]


In [318]:
for templ in ['inkjet']:
    for phone in phones:
        phone_data = data[(data.Device==phone)&(data.Strip=='contec')&(data.Template==templ)][indicators].iloc[:-2].values

        phone
        print phone_data
        

[[ 0.  2.  1.  0.  0.  1.  0.  5.  1.  3.]
 [ 0.  2.  1.  1.  0.  1.  0.  0.  0.  6.]
 [ 0.  0.  2.  0.  0.  1.  1.  0.  1.  6.]
 [ 0.  2.  1.  1.  0.  0.  0.  1.  0.  4.]
 [ 0.  2.  2.  0.  0.  1.  0.  1.  1.  5.]
 [ 0.  3.  0.  0.  0.  1.  0.  0.  0.  5.]]
[[ 0.  3.  1.  2.  0.  5.  0.  4.  0.  3.]
 [ 0.  3.  0.  0.  0.  5.  0.  0.  0.  6.]
 [ 0.  3.  0.  0.  0.  5.  0.  0.  1.  6.]
 [ 0.  2.  0.  0.  0.  5.  0.  0.  0.  6.]
 [ 0.  2.  1.  0.  0.  5.  0.  1.  0.  6.]
 [ 0.  4.  0.  0.  0.  5.  0.  0.  0.  5.]]
[[ 0.  2.  0.  2.  0.  0.  0.  5.  0.  3.]
 [ 0.  2.  1.  0.  0.  0.  0.  0.  0.  6.]
 [ 0.  2.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 0.  2.  1.  0.  0.  0.  0.  0.  0.  6.]
 [ 0.  2.  0.  0.  0.  0.  0.  1.  0.  6.]
 [ 0.  3.  1.  0.  0.  0.  0.  0.  0.  4.]]


In [334]:
for templ in template_types:
    for phone in phones:
        phone_data = data[(data.Device==phone)&(data.Strip=='contec')&(data.Template==templ)][indicators].iloc[:-2].values
        
        print templ,phone
        #print np.mean(phone_data == eye_data)
        print indicators.values
        print np.mean(phone_data==eye_data,axis=0).values.round(2)
        print 
        

inkjet Samsung
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.67  0.17  0.17  0.5   1.    0.17  0.5   0.5   0.5   0.5 ]

inkjet XiaomiB
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.67  0.17  0.33  0.67  1.    0.    0.33  0.17  0.83  0.83]

inkjet XiaomiD
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.67  0.    0.17  0.67  1.    1.    0.5   0.33  1.    0.83]

laser Samsung
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.67  0.    0.    0.67  1.    0.83  0.83  0.5   0.83  0.67]

laser XiaomiB
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.67  0.17  0.17  0.67  1.    0.    0.33  0.67  0.5   0.67]

laser XiaomiD
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.67  0.    0.17  0.67  1.    0.67  0.33  0.83  1.    0.67]



In [337]:
for templ in template_types:
    for phone in phones:
        phone_data = data[(data.Device==phone)&(data.Strip=='contec')&(data.Template==templ)][indicators].iloc[:-2].values

        print templ,phone
        print indicators.values
        print np.mean(np.abs(phone_data-eye_data)<=1,axis=0).values.round(2)
        

inkjet Samsung
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.33  0.67  0.83  1.    1.    1.    1.    1.    0.67]
inkjet XiaomiB
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.17  0.67  0.83  1.    0.    1.    1.    1.    0.83]
inkjet XiaomiD
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.17  0.83  0.83  1.    1.    1.    1.    1.    1.  ]
laser Samsung
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.33  0.67  0.67  1.    0.83  1.    1.    0.83  0.67]
laser XiaomiB
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.17  0.67  0.67  1.    0.    1.    1.    1.    0.83]
laser XiaomiD
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.17  0.67  0.83  1.    0.67  1.    1.    1.    0.83]


In [338]:
eye_data = data[(data.Device=='eye')][indicators]

In [345]:
real_data = data[(data.Device=='Sysmex')][indicators]

In [346]:
chinese_data = data[data.Device == 'chinese_device'][indicators]

In [348]:
print chinese_data

    URO  BLO  BIL  KET  LEU  GLU  PRO  PH  NIT   SG
13    0    0    1    3    0    0    0   5    0  3.0
15    0    0    0    0    0    0    0   1    0  6.0
14    0    0    1    0    0    0    0   1    0  6.0
11    0    0    0    0    0    0    0   1    0  6.0
12    0    0    0    0    0    0    0   1    0  6.0
10    0    0    0    0    0    0    0   1    0  6.0
8     0    0    0    2    0    0    0   3    0  3.0
9     0    0    2    0    0    5    3   1    0  4.0


In [349]:
print real_data

   URO  BLO  BIL  KET  LEU  GLU  PRO  PH  NIT   SG
5    1    0    0    2    0    0    1   6    0  5.0
7    0    0    0    2    0    0    1   0    0  5.0
6    0    0    0    1    0    0    2   1    0  6.0
3    0    0    0    0    1    0    1   2    0  4.0
4    1    0    0    0    0    0    0   2    0  4.0
2    0    4    0    0    0    0    0   1    0  3.0
0    2    0    0    3    3    0    1   5    0  4.0
1    0    5    2    0    0    3    3   2    2  3.0


In [351]:
print real_data.values == chinese_data.values

[[False  True False False  True  True False False  True False]
 [ True  True  True False  True  True False False  True False]
 [ True  True False False  True  True False  True  True  True]
 [ True  True  True  True False  True False False  True False]
 [False  True  True  True  True  True  True False  True False]
 [ True False  True  True  True  True  True  True  True False]
 [False  True  True False False  True False False  True False]
 [ True False  True  True  True False  True False False False]]


In [340]:
np.mean(real_data.values == eye_data.values)

0.71666666666666667

In [341]:
np.mean(np.abs(real_data.values - eye_data.values)<=1)

0.90000000000000002

In [342]:
print np.abs(real_data.values - eye_data.values)<=1

[[ True  True  True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True False  True  True  True  True  True  True False]
 [ True  True False  True  True  True  True  True  True False]
 [ True  True False  True  True  True  True  True  True  True]]


In [343]:
print real_data

   URO  BLO  BIL  KET  LEU  GLU  PRO  PH  NIT   SG
5    1    0    0    2    0    0    1   6    0  5.0
7    0    0    0    2    0    0    1   0    0  5.0
6    0    0    0    1    0    0    2   1    0  6.0
3    0    0    0    0    1    0    1   2    0  4.0
4    1    0    0    0    0    0    0   2    0  4.0
2    0    4    0    0    0    0    0   1    0  3.0


In [344]:
print eye_data

    URO  BLO  BIL  KET  LEU  GLU  PRO  PH  NIT   SG
19    1    0    0    3    0    0    1   5    0  3.0
21    0    0    0    2    0    0    0   1    0  6.0
20    0    0    0    0    0    0    1   1    0  6.0
17    0    0    2    0    0    0    1   1    0  6.0
18    1    0    2    0    0    0    1   1    0  6.0
16    0    4    2    0    0    0    0   1    0  3.0


###  Выводы:

- Нужна более разнородная выборка, объем выборки должен быть больше
- Нужно попробовать фотографировать со вспышкой
- Нужно определенным образом выбрать шаблон. Подходит ли этот? (Нет)

In [375]:
real_data = data[(data.Device=='Sysmex')&(data.Strip=='sysmex')][indicators].values[:,:-1]

In [376]:
phone_data = data[(data.Device==phone)&(data.Strip=='sysmex')&(data.Template==templ)]

In [377]:
phone_data.iloc[:-1]

,ID,URO,BLO,BIL,KET,LEU,GLU,PRO,PH,NIT,SG,Strip,Template,Device
39,10114450,0,2,0,0,0,3,0,5,0,NaN,sysmex,inkjet,Samsung
40,10114975,0,3,0,0,0,0,0,3,0,NaN,sysmex,inkjet,Samsung
41,10115025,0,2,1,0,1,3,1,2,1,NaN,sysmex,inkjet,Samsung
44,30305691,0,3,0,0,1,0,0,2,0,NaN,sysmex,inkjet,Samsung
43,30305695,0,3,0,0,0,0,0,3,0,NaN,sysmex,inkjet,Samsung
46,30305842,0,5,1,0,1,1,2,3,1,NaN,sysmex,inkjet,Samsung
42,check_1,2,0,1,2,2,3,1,6,1,NaN,sysmex,inkjet,Samsung
47,check_2,0,2,1,0,0,3,4,5,1,NaN,sysmex,inkjet,Samsung


In [389]:
templ = 'inkjet'
for phone in phones:
    phone_data = data[(data.Device==phone)&(data.Strip=='sysmex')&(data.Template==templ)][indicators].iloc[:-1,:-1].values
    print templ,phone
        #print np.mean(phone_data == eye_data)
    print indicators.values
    print np.mean(phone_data==real_data,axis=0).round(2)
    print np.mean(phone_data==real_data)
        

inkjet Samsung
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.75  0.12  0.5   0.5   0.62  0.5   0.25  0.12  0.5 ]
0.430555555556
inkjet XiaomiB
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.62  0.    0.25  0.5   0.38  0.12  0.12  0.    0.  ]
0.222222222222
inkjet XiaomiD
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 0.75  0.    0.62  0.5   0.62  0.    0.62  0.12  0.  ]
0.361111111111


In [386]:
templ = 'inkjet'
for phone in phones:
    phone_data = data[(data.Device==phone)&(data.Strip=='sysmex')&(data.Template==templ)][indicators].iloc[:-1,:-1].values
    print templ,phone
        #print np.mean(phone_data == eye_data)
    print indicators.values
    print np.mean(np.abs(phone_data-real_data)<=1,axis=0).round(2)
    print np.mean(np.abs(phone_data-real_data)<=1)

inkjet Samsung
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.25  1.    0.75  1.    0.62  0.88  0.62  1.  ]
0.791666666667
inkjet XiaomiB
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.    0.75  0.88  0.88  0.12  0.62  0.62  0.88]
0.638888888889
inkjet XiaomiD
['URO' 'BLO' 'BIL' 'KET' 'LEU' 'GLU' 'PRO' 'PH' 'NIT' 'SG']
[ 1.    0.12  0.88  0.75  0.88  0.12  0.88  0.62  1.  ]
0.694444444444
